In [ ]:
y_obs = sunsopts_num

# Modello Bayesian con Turing
@model function bayesian_model(y_obs)
    N ~ Uniform(0, 10)
    q ~ Uniform(0, 10)
    tau = 1

    # Condizioni iniziali e setup ODE
    tspan = (0.0, length(y_obs))
    p = [N, q, tau]
    
    # Risolvi l'ODE
    prob = DDEProblem(MagneticField, B0, h, tspan, p)
    solution = solve(prob, MethodOfSteps(Tsit5()), saveat = 0.01)
    B_solution = [sol[1] for sol in solution.u]

    # Likelihood
    for i in 1:length(y_obs)
        y_obs[i] ~ Normal(B_solution[i], 1.0)
    end
end

# Esecuzione MCMC
model = bayesian_model(y_obs)
chain = sample(model, NUTS(), 100)